<a href="https://www.kaggle.com/code/houmannorasteh/mpqa-on-srl?scriptVersionId=171511269" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
! pip install transformers
! pip install torch

### importing libraries 

In [2]:
import torch
import numpy as np
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
import json
from itertools import zip_longest

### defining general variables

In [3]:
#defining global valriables throughout the whole notebook
EPOCH = 10
BATCH_SIZE = 16
MAX_INPUT_LENGTH = 65
MAX_LABEL_LENGTH = 8
MODEL_LINK = "google/flan-t5-small"

In [4]:
tokenizer = T5Tokenizer.from_pretrained(MODEL_LINK, legacy=False)
model = T5ForConditionalGeneration.from_pretrained(MODEL_LINK).to('cuda')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### functions created for reading(get_data) and organize the files (organize_data)

In [ ]:
# a funciton to read data off of a database link is here to help getting and organizing data into dataframes
def get_data(address):
    lines = []
    with open(address) as file:
        for line in file:
            x = json.loads(line)
            lines.append(x)
    sentences, orl, sep_sentences = [], [], []
    for i in range(len(lines)):
        sep_sentences.append(lines[i]['sentences'])
        sentences.append(' '.join(lines[i]['sentences']))
        orl.append(lines[i]['orl'])
    dataframe = pd.DataFrame({'sentence': sentences, 'orl': orl, 'sep_sent': sep_sentences})
    return dataframe

# this function is to make a list of the said attribute for later iterations
def list_of(attributes, requested_atr):
    requested_list = []
    for sublist in attributes:
        if sublist[-1] == requested_atr:
            requested_list.append(sublist)
    return requested_list

# this function was made to find target(s)/agent(s) of a dse according to list of attributes
def organize_data(attributes, sentence):
    AGENT, DSE, TARGET = '', '', ''
    target_flag, agent_flag = False, False
    for sublist in attributes:
        if sublist[-1] == 'DSE':
            dse_start = int(sublist[0])
            dse_end = int(sublist[1] + 1)
            DSE += ' '.join(sentence[dse_start:dse_end]) + '|'
            
            
            # looking for the targets and agents of this dse that we have found
            target_flag = False
            for sub_sublist in list_of(attributes, 'TARGET'):
                if sub_sublist[0] == dse_start and int(sub_sublist[1] + 1) == dse_end:
                    target_start = int(sub_sublist[2])
                    target_end = int(sub_sublist[3] + 1)
                    TARGET += ' '.join(sentence[target_start:target_end]) + '|'
                    target_flag = True
            if not target_flag:
                TARGET += ' |'
            
            agent_flag = False
            for sub_sublist in list_of(attributes, 'AGENT'):
                if sub_sublist[0] == dse_start and int(sub_sublist[1] + 1) == dse_end:
                    agent_start = int(sub_sublist[2])
                    agent_end = int(sub_sublist[3] + 1)
                    AGENT += ' '.join(sentence[agent_start:agent_end]) + '|'
                    agent_flag = True
            if not agent_flag:
                AGENT += ' |'
    return AGENT, DSE, TARGET

### a function to call for different links of folders with exact process to extract and organize data

In [ ]:
def get_files_of_folder(folder_number):
    folder_number = int()
    folder = f"/kaggle/input/ds-json-format/json_format_dataset/{folder_number}"
    dev_df = get_data(f"{folder}/aaai19srl.dev{folder_number}.conll.json")
    df = get_data(f"{folder}/aaai19srl.train{folder_number}.conll.json")
    test_df = get_data(f"{folder}/aaai19srl.test{folder_number}.conll.json")
    return df, dev_df, test_df
df = get_files_of_folder(4)[0]
dev_df = get_files_of_folder(4)[1]
test_df = get_files_of_folder(4)[2]

### getting training/validation/test data into data frames and dividing each of {agent, target, dse} elements

In [ ]:
# df = get_data("/kaggle/input/ds-json-format/json_format_dataset/0/aaai19srl.train0.conll.json")

for i in range(len(df)):
    agent, dse, target = organize_data(df['orl'][i], df['sep_sent'][i])
    df.loc[i, 'agent'] = agent
    df.loc[i, 'dse'] = dse
    df.loc[i, 'target'] = target

df.drop(columns='sep_sent')

In [ ]:
# dev_df = get_data("/kaggle/input/ds-json-format/json_format_dataset/0/aaai19srl.dev0.conll.json")

for i in range(len(dev_df)):
    agent, dse, target = organize_data(dev_df['orl'][i], dev_df['sep_sent'][i])
    dev_df.loc[i, 'agent'] = agent
    dev_df.loc[i, 'dse'] = dse
    dev_df.loc[i, 'target'] = target

dev_df.drop(columns='sep_sent')

In [ ]:
# test_df = get_data("/kaggle/input/ds-json-format/json_format_dataset/0/aaai19srl.test0.conll.json")

for i in range(len(test_df)):
    agent, dse, target = organize_data(test_df['orl'][i], test_df['sep_sent'][i])
    test_df.loc[i, 'agent'] = agent
    test_df.loc[i, 'dse'] = dse
    test_df.loc[i, 'target'] = target

test_df.drop(columns='sep_sent')

### Getting evaluation data into df and dividing each of {agent, target, dse} elements

In [ ]:
def pipeDivider(pipedString):
    listOfItems = []
    listOfItems = pipedString.split('|')[:-1]
    return listOfItems

In [ ]:
# function which will write prompt for the model according to the sentence and the items in it
def create_prompt(input_df):
    output_df = pd.DataFrame(columns=['target_prompt', 'agent_prompt', 'target', 'agent'])
    for i in range(len(input_df)):
        dse_list, target_list, agent_list = [], [], []
        dse_list = pipeDivider(str(input_df.iloc[i]['dse']))
        target_list = pipeDivider(str(input_df.iloc[i]['target']))
        agent_list = pipeDivider(str(input_df.iloc[i]['agent']))
        for j in range(len(dse_list)):
            last_row = int(len(output_df))+1
            output_df.loc[last_row, 'target_prompt'] = f"Sentence is: {input_df.iloc[i]['sentence']} Find target for this dse: {dse_list[j]}"
            output_df.loc[last_row, 'agent_prompt'] = f"Sentence is: {input_df.iloc[i]['sentence']} Find agent for this dse: {dse_list[j]}"
            output_df.loc[last_row, 'target'] = target_list[j]
            output_df.loc[last_row, 'agent'] = agent_list[j]
    return output_df

In [ ]:
train_df = create_prompt(df)
train_df.head(-10)

### Change the sentence and dse into a prompt according to information in that row 
#### change will be applied on all three data frames {train, test, validation}

In [ ]:
dev_df = create_prompt(dev_df)
train_df = create_prompt(df)
test_df = create_prompt(test_df)

In [ ]:
train_df = train_df.reset_index(drop=True)
dev_df = dev_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [5]:
from torch.utils.data import DataLoader, Dataset
# Define a custom dataset
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length4text, max_length4label):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length4text = max_length4text
        self.max_length4label = max_length4label

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        agent_prompt = self.data['agent_prompt'][idx]
        target_prompt = self.data['target_prompt'][idx]
        agent = self.data['agent'][idx]
        target = self.data['target'][idx]
        # tokenizing agent prompt
        agent_prompt_encoding = self.tokenizer(agent_prompt, truncation=True, padding='max_length', max_length=self.max_length4text, return_tensors='pt').to("cuda")
        # tokenizing target prompt
        target_prompt_encoding = self.tokenizer(target_prompt, truncation=True, padding='max_length', max_length=self.max_length4text, return_tensors='pt').to("cuda")
        # tokenizing agent
        agent_encoding = self.tokenizer(agent, truncation=True, padding='max_length', max_length=self.max_length4label, return_tensors='pt').to("cuda")
        # tokenizing target
        target_encoding = self.tokenizer(target, truncation=True, padding='max_length', max_length=self.max_length4label, return_tensors='pt').to("cuda")
        return {
            # agent prompt
            'agent_input_id': agent_prompt_encoding['input_ids'].squeeze(),
            'agent_attention_mask': agent_prompt_encoding['attention_mask'].squeeze(),
            # target prompt
            'target_input_id': target_prompt_encoding['input_ids'].squeeze(),
            'target_attention_mask': target_prompt_encoding['attention_mask'].squeeze(),
            #agent
            'agent_id': agent_encoding['input_ids'].squeeze(),
            'agent_mask': agent_encoding['attention_mask'].squeeze(),
            #target
            'target_id': target_encoding['input_ids'].squeeze(),
            'target_mask': target_encoding['attention_mask'].squeeze()
        }

In [ ]:
# Create a data loader for TRAIN dataframe 
train_dataset = CustomDataset(train_df, tokenizer, max_length4text= MAX_INPUT_LENGTH, max_length4label= MAX_LABEL_LENGTH)
train_data_loader = DataLoader(train_dataset, batch_size= BATCH_SIZE, shuffle=True)

# Create a data loader for EVALUATION dataframe
val_dataset = CustomDataset(dev_df, tokenizer, max_length4text= MAX_INPUT_LENGTH, max_length4label= MAX_LABEL_LENGTH)
val_data_loader = DataLoader(val_dataset, batch_size= BATCH_SIZE, shuffle=False)

# Create a data loader for TEST dataframe
test_dataset = CustomDataset(test_df, tokenizer, max_length4text= MAX_INPUT_LENGTH, max_length4label= MAX_LABEL_LENGTH)
test_data_loader = DataLoader(test_dataset, batch_size= BATCH_SIZE, shuffle=False)



In [14]:
# Define the loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [6]:
def f1_calculator(pred_list, actual_list):
    cleaned_pred, cleaned_actual = [], []
    matched = 0
    for i in range(len(pred_list)):
        for j in range(len(actual_list)):
            if actual_list[j] in pred_list:
                matched += 1
        cleaned_pred.extend(pred_list)
        cleaned_actual.extend(actual_list)
    
    prediction_len = len(cleaned_pred)
    actual_len = len(cleaned_actual)
    print(f'matched: {matched}, prediction_len:{prediction_len}, actual_len:{actual_len} \n')
    try:
        precision = (matched / prediction_len)
        recall = (matched / actual_len)
        f1 = (2 * (precision * recall)) / (precision + recall)
    except:
        f1 = 0
    return f1

In [7]:
def evaluate_model(model, dataloader, prompt_type):
    if prompt_type == 'target':
        id_type = 'target_input_id'
        attention_type = 'target_attention_mask'
        output_type = 'target_id'
    elif prompt_type == 'agent':
        id_type = 'agent_input_id'
        attention_type = 'agent_attention_mask'
        output_type = 'agent_id'
    
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        true_counted, prediction_counted, actual_counted = 0,0,0
        matched, prediction_len, actual_len = 0,0,0
        actual_list, prediction_list, f1 = [], [], []
        for batch_idx, batch in enumerate(dataloader):
            # Move data to the specified device
            # batch = {key: value.to('cuda') for key, value in batch.items()}

            # Forward pass
            ids = batch[id_type]
            mask = batch[attention_type]
            output_id = batch[output_type]
            
            actuals = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in output_id]
            
            generated_output = generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=64, 
              )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_output]
            
            actual_list.extend(actuals)
            prediction_list.extend(preds)
    return f1_calculator(prediction_list, actual_list)

In [17]:
def train_data(data_loader, v_data_loader):
    model.train()
    for epoch in range(EPOCH):
        losses = []
        for batch in data_loader:

            agent_input = batch['agent_input_id']
            agent_attention_mask = batch['agent_attention_mask']

            target_input = batch['target_input_id']
            target_attention_mask = batch['target_attention_mask']

            agent_id = batch['agent_id']
            agent_mask = batch['agent_mask']

            target_id = batch['target_id']
            target_mask = batch['target_mask']

            optimizer.zero_grad()

            agent_output = model(agent_input, attention_mask=agent_attention_mask, labels=agent_id)
            target_output = model(target_input, attention_mask=target_attention_mask, labels=target_id)

            agent_loss = agent_output.loss
            target_loss = target_output.loss
            losses.append(agent_loss.item())
            losses.append(target_loss.item())

            agent_loss.backward()
            target_loss.backward()
            optimizer.step()

        f1_4_target = evaluate_model(model, v_data_loader, 'target')
        f1_4_agent = evaluate_model(model, v_data_loader, 'agent')
        print(f'loss: {np.mean(losses)}, f1 for target:{f1_4_target}, f1 for agent:{f1_4_agent} \n end of epoch{epoch}. \n')

In [ ]:
# for folder 1:
train_data(train_data_loader)

### Here we can either use the model we trained above, or we can use the link to the same model.

In [ ]:
# # defining a new model to fit the state dictionary of the previously trained model to it.
# new_model = T5ForConditionalGeneration.from_pretrained(MODEL_LINK).to('cuda')

# state_dict = torch.load("/kaggle/input/t5-small_trained_mpqa/pytorch/t5-small_mpqa_v1/1/t5_orl_model.pt")
# new_model.load_state_dict(state_dict)

### Here we are gonna load the test dataset into evaluation method to calculate score

### Applying 10 percent of SRL to the trained data to see how good or bad does it perform

##### importing and cleaning data for training

In [9]:
srl = pd.read_csv('/kaggle/input/10-of-srl/10_percent.csv')
agents, targets, verbs, sentences = [], [], [], []
for i in range(len(srl)):
    line = srl.iloc[i]
    temp = line['arguments'].split('|')
    agents.append(temp[0])
    targets.append(temp[1])
    sentences.append(line['sentence'])
    verbs.append(line['predicate'])

srl_df = pd.DataFrame({'sentences': sentences, 'verbs': verbs, 'agents': agents, 'targets': targets})
srl_df.head()

,sentences,verbs,agents,targets
0,"Pierre Vinken, 61 years old, will join the boa...",join,"Pierre Vinken, 61 years old,",the board
1,"Mr. Vinken is chairman of Elsevier N.V., the D...",publishing,group,
2,"Rudolph Agnew, 55 years old and former chairma...",named,,"Rudolph Agnew, 55 years old and former chairma..."
3,A form of asbestos once used * * to make Kent ...,used,,A form of asbestos *
4,A form of asbestos once used * * to make Kent ...,make,*,Kent cigarette filters


##### in this format which data is ready to be used, we make appropriate changes (like adding prompt) to feed it to the data loader for later usage.

In [10]:
agent_prompts, agents, target_prompts, targets = [], [], [], []
for i in range(len(srl_df)):
    line = srl_df.iloc[i]
    agent_prompts.append(f" sentence is: {line['sentences']} find agent for this verb: {line['verbs']}")
    agents.append(line['agents'])
    target_prompts.append(f" sentence is: {line['sentences']} find target for this verb: {line['verbs']}")
    targets.append(line['targets'])
srl_df = []
srl_df = pd.DataFrame({'agent_prompt': agent_prompts, 'agent': agents, 'target_prompt': target_prompts, 'target': targets})
srl_df

,agent_prompt,agent,target_prompt,target
0,"sentence is: Pierre Vinken, 61 years old, wil...","Pierre Vinken, 61 years old,","sentence is: Pierre Vinken, 61 years old, wil...",the board
1,sentence is: Mr. Vinken is chairman of Elsevi...,group,sentence is: Mr. Vinken is chairman of Elsevi...,
2,"sentence is: Rudolph Agnew, 55 years old and ...",,"sentence is: Rudolph Agnew, 55 years old and ...","Rudolph Agnew, 55 years old and former chairma..."
3,sentence is: A form of asbestos once used * *...,,sentence is: A form of asbestos once used * *...,A form of asbestos *
4,sentence is: A form of asbestos once used * *...,*,sentence is: A form of asbestos once used * *...,Kent cigarette filters
...,...,...,...,...
9347,sentence is: Trinity Industries Inc. said 0 i...,it,sentence is: Trinity Industries Inc. said 0 i...,a preliminary agreement * to sell 500 railcar ...
9348,sentence is: Trinity Industries Inc. said 0 i...,it *,sentence is: Trinity Industries Inc. said 0 i...,500 railcar platforms
9349,sentence is: Terms weren't disclosed *-1. fin...,,sentence is: Terms weren't disclosed *-1. fin...,Terms *-1
9350,sentence is: Trinity said 0 it plans *-1 to b...,Trinity,sentence is: Trinity said 0 it plans *-1 to b...,0 it plans *-1 to begin delivery in the first ...


In [11]:
# Sample size (60% of total rows)
train_size = int(0.6 * len(srl_df))
test_size = int(0.2 * len(srl_df))


# Select 60% of the DataFrame randomly
srl_df_sampled = srl_df.sample(sample_size, random_state=42)

srl_train_dataset = CustomDataset(srl_df, tokenizer, max_length4text= MAX_INPUT_LENGTH, max_length4label= MAX_LABEL_LENGTH)
srl_train_data_loader = DataLoader(srl_train_dataset, batch_size= BATCH_SIZE, shuffle=True)

srl_test_dataset = 

In [16]:
train_data(srl_train_data_loader, )

NameError: name 'val_data_loader' is not defined

In [ ]:
target_accuracy = evaluate_model(model, test_data_loader, 'target')
agent_accuracy = evaluate_model(model, test_data_loader, 'agent')
print(f'f1 Agent: {agent_accuracy}. f1 Target: {target_accuracy}\n\n----------------------')